In [658]:
import sgp4.api
import sys
import json
import base64
import requests
import re
import math
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from sgp4.api import Satrec
from sgp4.api import jday

In [666]:
response = requests.get("https://ssd.jpl.nasa.gov/api/horizons.api?COMMAND='10'&EPHEM_TYPE='VECTORS'&REF_PLANE='F'&START_TIME='2022-Jan-01 00:00:00'&STOP_TIME='2022-Jan-01 01:30:00'&STEP_SIZE='1 min'")
file = json.loads(response.text)
sun = file['result']
sun1 = sun[sun.find('$$SOE')+len('$$SOE'):sun.find('$$EOE')]
with open('sunvectors.txt','w') as ss:
    snvecs = ss.write(sun1)
    ss.close()
print(sun)

*******************************************************************************
 Revised: July 31, 2013                  Sun                                 10

 PHYSICAL PROPERTIES (updated 2018-Aug-15):
  GM, km^3/s^2          = 132712440041.93938  Mass, 10^24 kg        = ~1988500
  Vol. mean radius, km  = 695700              Volume, 10^12 km^3    = 1412000
  Solar radius (IAU)    = 696000 km           Mean density, g/cm^3  = 1.408
  Radius (photosphere)  = 696500 km           Angular diam at 1 AU  = 1919.3"
  Photosphere temp., K  = 6600 (bottom)       Photosphere temp., K  = 4400(top)
  Photospheric depth    = ~500 km             Chromospheric depth   = ~2500 km
  Flatness, f           = 0.00005             Adopted sid. rot. per.= 25.38 d
  Surface gravity       =  274.0 m/s^2        Escape speed, km/s    =  617.7
  Pole (RA,DEC), deg.   = (286.13, 63.87)     Obliquity to ecliptic = 7.25 deg.
  Solar constant (1 AU) = 1367.6 W/m^2        Luminosity, 10^24 J/s = 382.8
  Mass-energy 

In [676]:
#Getting sun position vector from JPL Horizons data
with open('sunvectors.txt') as sx:
    sunvec = sx.read().splitlines()
cks = [sunvec[n:n+4] for n in range(0,len(sunvec),4)]
numeric_const_pattern = '[-+]? (?: (?: \d* \. \d+ ) | (?: \d+ \.? ) )(?: [Ee] [+-]? \d+ ) ?'
rx = re.compile(numeric_const_pattern, re.VERBOSE)
q = rx.findall(str(cks))
jd = []
x1 = []
x2 = []
x3 = []
for i in range(0,len(cks)):
    jd.append(q[i])
    x1.append(q[i+6])
    x2.append(q[i+7])
    x3.append(q[i+8])
sv = [0]*len(cks)
for i in range(0,len(cks)):
    sv[i] = (x1[i], x2[i], x3[i])
sv = np.array(sv, dtype=float)
sv = sv*1000
print(sv)
norm_sv = (np.linalg.norm(sv,axis=1))
normed_sv = [0]*len(cks)
for i in range(0,len(cks)):
    normed_sv[i] = sv[i]/norm_sv[i]
normed_sv = np.array(normed_sv, dtype=float)
print(normed_sv)
magsv = np.linalg.norm(normed_sv,axis=1)

[[ 2.61278010e+10 -1.32825709e+11 -5.75795605e+10]
 [-1.32825709e+11 -5.75795605e+10  2.98122059e+04]
 [-5.75795605e+10  2.98122059e+04  4.95583766e+03]
 [ 2.98122059e+04  4.95583766e+03  2.14695131e+03]
 [ 4.95583766e+03  2.14695131e+03  4.90699165e+05]
 [ 2.14695131e+03  4.90699165e+05  1.47107909e+11]
 [ 4.90699165e+05  1.47107909e+11 -2.00926297e+01]
 [ 1.47107909e+11 -2.00926297e+01  2.45958050e+09]
 [-2.00926297e+01  2.45958050e+09  2.02200000e+06]
 [ 2.45958050e+09  2.02200000e+06 -1.00000000e+03]
 [ 2.02200000e+06 -1.00000000e+03  0.00000000e+00]
 [-1.00000000e+03  0.00000000e+00  1.00000000e+03]
 [ 0.00000000e+00  1.00000000e+03  0.00000000e+00]
 [ 1.00000000e+03  0.00000000e+00  2.61295897e+10]
 [ 0.00000000e+00  2.61295897e+10 -1.32825412e+11]
 [ 2.61295897e+10 -1.32825412e+11 -5.75794317e+10]
 [-1.32825412e+11 -5.75794317e+10  2.98121411e+04]
 [-5.75794317e+10  2.98121411e+04  4.95617191e+03]
 [ 2.98121411e+04  4.95617191e+03  2.14709626e+03]
 [ 4.95617191e+03  2.14709626e+

In [688]:
#Getting Starlink position vector from sgp4 (equatorial)
with open('starlinktle.txt') as fn:
    sl = fn.read().splitlines() 
ch = [sl[n:n + 3] for n in range(0, len(sl), 3)]
print("There are " + str(len(ch)) + " satellites")
for c in ch[:7]:
    sat_num = c[0].strip()  
    s = c[1]  
    t = c[2]  
    sat = Satrec.twoline2rv(s, t)  
fr = 0.0
e = [0]*len(cks)
r = [0]*len(cks)
v = [0]*len(cks)
for i in range(0,len(cks)):
    e[i], r[i], v[i] = sat.sgp4(float(jd[i]),fr)
e = np.array(e, dtype=float)
r = np.array(r, dtype=float)
v = np.array(v, dtype=float)
r = r*1000
print(r)
norm_r = (np.linalg.norm(r,axis=1))
normed_r = [0]*len(cks)
for i in range(0,len(cks)):
    normed_r[i] = r[i]/norm_r[i]
normed_r = np.array(normed_r, dtype=float)
print(normed_r)
magr = np.linalg.norm(normed_r,axis=1)

There are 2623 satellites
[[ 6.37325523e+06  1.75146781e+06  2.06580572e+06]
 [-6.50673848e+19 -4.82197259e+19 -8.52266858e+19]
 [ 9.33319419e+19 -7.37358779e+19  6.02261778e+17]
 [ 1.06273657e+20  4.25885942e+19  3.29544254e+19]
 [ 1.06273657e+20  4.25885942e+19  3.29544254e+19]
 [ 1.06273657e+20  4.25885942e+19  3.29544254e+19]
 [            nan             nan             nan]
 [            nan             nan             nan]
 [-4.59581472e+29  1.28241374e+31 -2.38829342e+30]
 [ 8.72279802e+19 -4.74003081e+19 -5.53888666e+19]
 [ 1.63818575e+19 -9.48402742e+19  7.01744136e+19]
 [-1.48483779e+19 -9.40402540e+19  6.53404335e+19]
 [-3.09783863e+19  1.09951927e+20  3.06592142e+19]
 [            nan             nan             nan]
 [ 3.61980680e+19 -6.30326743e+19 -8.99328564e+19]
 [ 6.38566674e+06  2.05200379e+06  1.72393526e+06]
 [-6.50673848e+19 -4.82197259e+19 -8.52266858e+19]
 [ 9.33319419e+19 -7.37358779e+19  6.02261778e+17]
 [ 1.06273657e+20  4.25885942e+19  3.29544254e+19]
 [-4.

In [681]:
#Brightness model 
#Nightside/Antisolar model 
M_sun = -26.74 
a_sp = 0.02 # diffuse albedo of solar panel 
A_sp = 30 # cross-sectional area of solar panel 
a_bus = 0.7 # albedo of cylindrical bus 
A_bus = 0.2*1.05 # cross-sectional area of cylindrical bus 
R = 1000*1000 # range to satellite 
R_e = 6378*1000
dot = [0]*len(cks)
phi = [0]*len(cks)
for i in range(0,len(cks)):
    dot[i] = np.dot(normed_r[i],normed_sv[i])
val = dot/(magr*magsv)
for i in range(0,len(cks)):
    phi[i] = math.acos(val[i])
print(phi)
phi_fn = [0]*len(cks)
M1 = [0]*len(cks)
for i in range(0,len(cks)):
    if phi[i]<1.5708:
        phi_fn[i] = ((math.pi-phi[i])*math.cos(phi[i])+math.sin(phi[i]))/(4*math.pi)
        M1[i] = M_sun - 2.5*math.log10((a_sp*A_sp*abs(math.cos(phi[i])))/(math.pi*R*R)) - 2.5*math.log10((a_bus*A_bus*phi_fn[i])/(R*R))                           
print(M1)

[1.7534824460642755, 0.8353645621309863, 2.472939339329177, 0.29840220213690766, 1.2795328249516305, 1.2905334433650624, nan, nan, 0.18823241538064486, 0.696706112118573, 1.4324273622680432, 1.0574986044286052, 0.37739912647397533, nan, 0.8527809061656575, 1.7733707254402442, 0.8353645620196795, 2.472939339329161, 0.2983909031949384, 2.159844032141428, 1.2905334433641475, nan, nan, 1.0313399024076264, 0.8904399835032842, 1.556774054548299, 0.6381153595312614, 0.8796906567136396, nan, 0.8367845882150593, 1.7924453963103402, 0.8353645619083769, 2.472939339329146, 0.2983796043172258, 2.274021342370337, 1.290533443363233, nan, nan, 1.7412536446668347, 2.418157593893199, 1.8834446968988174, 2.375001403200587, 2.063612675194612, nan, 1.0598133266972445, 1.8106209535667601, 0.835364561797078, 2.4729393393291303, 0.2983683055037813, 0.7757056710763703, 1.2905334433623181, nan, nan, 1.882245897797638, 1.3424280958897377, 0.8709014479701144, 1.6329150380635535, 2.080297861849773, nan, 1.60815183

In [559]:
#Satellite sub-point calculation
R_cir = 6371*10
h = 550*1000    
rx = [0]*15
ry = [0]*15
rz = [0]*15
for i in range(0,15):
    rx[i] = r[i,0]
    ry[i] = r[i,1]
    rz[i] = (r[i,2])
value = [0]*15
for i in range(0,15):
    value[i] = rz[i]/(np.sqrt(rx[i]**2+ry[i]**2))
hh = [0]*15
Rx = [0]*15
thetax = [0]*15
C = [0]*15
thetax = np.array(thetax)
for i in range(0,15):
    if ((value[i]>-1) and (value[i]<1)):
        thetax[i] = math.acos(rz[i]/(np.sqrt(rx[i]**2+ry[i]**2)))
    hh[i] = np.sqrt(rx[i]**2+ry[i]**2+rz[i]**2) - R_cir
    Rx[i] = np.sqrt((hh[i]+R_cir)**2 - h**2)
x = [0]*15
flat = 1/298.26
ex = 2*flat - flat*flat
theta = thetax
thetai = theta
for i in range(0,15):
    j = 0;
    if abs(theta[j]-thetai[j])<1e-03:
        thetai[j] = theta[j]
        C[j] = 1/np.sqrt(1-ex*ex*math.sin(thetai[j])*math.sin(thetai[j]))
        theta[j] = math.atan((hh[j]+R_e*C[j]*ex*ex*math.sin(thetai[j]))/Rx[j])
        j = j+1
    x[i] = Rx[i]/math.cos(theta[i]) - R_e*C[i]
x = np.array(x, dtype=float)
print(x)

[   531683.47602777   6899270.92504365  12786351.71480782
 -16599800.74167329   6898916.37673669 -16605146.69215905
  12778521.82720053  12776200.44113953   6909844.01452534
   6899876.86204094  12785114.89008974 -16600874.57119959
   6898601.68798795 -16604834.3940532   12780079.61472599]


In [560]:
#Dayside model
h = 550*1000     # satellite's LEO altitude
H = h/R_e
a_e = 0.3        # Earth's albedo
a_spb = 0.35     # albedo of solar panel space-facing side
A_spb = 30
D_bus = 2.1
h_bus = 0.1

#xi = 0 - x/R_e     #depth of terminator
X = [0]*15
Q1 = [0]*15
Q = [0]*15
for i in range(0,15):
    X[i] = x[i]/h         # x = sub-satellite point to the Earth's terminator
    Q1[i] = 2.54*(1 - math.exp(X[i]/1.1))
    Q[i] = 4.84/(1 + math.exp(1.466*X[i]))
M2 = [0]*15
for i in range(0,15):
    if phi[i]>1.5708:
        phi_fn[i] = ((math.pi-phi[i])*math.cos(phi[i])+math.sin(phi[i]))/(4*math.pi)
        M2[i] = M_sun - 2.5*math.log10(a_e*abs(dot[i])/math.pi) - 2.5*math.log10(abs(Q1[i])) - 2.5*math.log10((a_spb*A_spb*math.cos(phi[i]-math.pi))/(R*R*math.pi)) - 2.5*math.log10(Q[i]) - 2.5*math.log10((D_bus*h_bus*a_bus)/(R*R*math.pi)) - 2.5*math.log10((a_bus*A_bus*phi_fn[i])/(R*R*math.pi))
print(M2)

[29.906311119489494, 0, 0, 28.68859520390474, 0, 29.20492050266757, 44.316684269881996, 0, 36.13612371109679, 0, 0, 28.644593776712107, 0, 29.398435747058613, 43.8923327733846]


In [663]:
jd = np.array(jd, dtype=float)
times = Time(jd, format='jd', scale = 'tdb')
print(times)

[2459580.91666667 2459580.95833333 2459581.         2459581.04166667
 2459581.08333333 2459581.125      2459581.16666667 2459581.20833333
 2459581.25       2459581.29166667 2459581.33333333 2459581.375
 2459581.41666667 2459581.45833333 2459581.5       ]


In [686]:
F0 = 1377
a = 1000*1000
A = 30
phi = np.array(phi,dtype=float)
F = [0]*len(cks)
m = [0]*len(cks)
for i in range(0,len(cks)):
    F[i] = F0*A*a_sp*math.cos(phi[i])/(a*a)
    m[i] = M_sun - 2.5*math.log10(abs(F[i])/F0)    
print(m)

[5.6664039971458315, 4.247965435899065, 4.077921925060576, 3.8636958446715397, 5.16930165479031, 5.209958915756854, nan, nan, 3.833971143639296, 4.102690066712892, 5.9654919393289525, 4.586801712409443, 3.8938509737632323, nan, 4.269236422891886, 5.555596444129758, 4.247965435765494, 4.07792192506059, 3.8636920713785514, 4.452775943020438, 5.209958915753404, nan, nan, 4.5379120316874655, 4.317874322515493, 8.447611467558342, 4.05253581745162, 4.303609549506756, nan, 4.249671963552945, 5.45936176105668, 4.247965435631926, 4.077921925060604, 3.86368829825874, 4.287896686865718, 5.209958915749954, nan, nan, 5.740845653042566, 4.127640237685419, 5.094727717651164, 4.170869321140582, 4.627218906654395, nan, 4.591272198063347, 5.3753152983158365, 4.247965435498362, 4.077921925060615, 3.8636845253121095, 4.180487224882729, 5.209958915746501, nan, nan, 5.098762524914672, 5.427486343289079, 4.2921757820845485, 6.83226413222798, 4.594146144932722, nan, 7.383987840643396, 5.301696573204506, 4.247